Imported some useful libraries and read our .csv file.

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate

df = pd.read_csv('player-stats-raw.csv')

I wasn't sure if there could have been any duplicates, but just in case I wrote this to drop any duplicate usernames that might appear.
Additionally, I renamed the timestamp column from "ts" to "timestamp" and converted it from an object type to a datetime type.

In [27]:
df = df.drop_duplicates(subset='username', keep='first')
df = df.rename(columns={'ts':'timestamp'})
df['timestamp'] = pd.to_datetime(df['timestamp'])

Right now I'm just interested in PvM (Player versus Monster) content, so I first dropped any rows that didn't end in "_score" which essentially just dropped all the stats related to player levels. Afterwards I manually dropped things like Bounty Hunter and Treasure Trails to leave us just with bossing content.

In [28]:
boss = df.filter(regex='_score')
boss = boss.drop(columns=['unknown_score','bounty_hunter_hunter_score','bounty_hunter_rogue_score','clue_scrolls_all_score','clue_scrolls_beginner_score',
'clue_scrolls_easy_score','clue_scrolls_medium_score','clue_scrolls_hard_score','clue_scrolls_elite_score','clue_scrolls_master_score','lms_rank_score',
'pvp_arena_score','soul_wars_zeal_score','rifts_closed_score'])

boss_list = list(boss.columns.values)

One last bit of preprocessing here, creating a list of all of the columns I want from the dataframe for my purposes and then using that list to filter our original dataframe to end up with a list solely comprised of each user and how many of each boss they've defeated.

The OSRS Hiscores work in a way that some bosses have a minimum required amount that needs to be defeated before the player gets listed on the Hiscores and their kill count is displayed. So in our case, any values of "-1" in our data means that the player either hasn't defeated that boss at all, or they have yet to reach the minimum required amount. 
What I opted to do is replace all values of -1 with a "Not listed on Hiscores." string.

Lastly, I dropped the "_score" from the end of each of the bosses and any other underscores in the columns just to make it look nicer.

In [29]:
col_list = ['username','timestamp'] + boss_list
player_kc = df.filter(col_list)
player_kc = player_kc.replace(-1,'Not listed on Hiscores.')
player_kc.columns = player_kc.columns.str.replace('_score','')
player_kc.columns = player_kc.columns.str.replace('_',' ')

Now what we've been waiting for:
I set this up to accept a case-sensitive username input from the user. In our case, we'll look at the user "Suede". (itswill7)
That username is then used to filter our "player_kc" dataframe to just give us our kill count stats for that particular player.

I transposed the dataframe at this point because it was much smaller and cause the runtimes to drop dramatically.
This leaves us with a table of kill counts for each boss with a string indicating if a particular boss has not been slain enough to list the player on the Hiscores.

In [30]:
name = input("Enter username: ")
user_kc = player_kc[player_kc['username'] == name]
user_kc = np.transpose(user_kc).reset_index()
user_kc['index'] = user_kc['index'].str.title()
print(tabulate(user_kc, tablefmt='fancy_grid', showindex=False))

╒══════════════════════════════════╤════════════════════════════╕
│ Username                         │ Suede                      │
├──────────────────────────────────┼────────────────────────────┤
│ Timestamp                        │ 2022-07-21 05:30:18.623315 │
├──────────────────────────────────┼────────────────────────────┤
│ Abyssal Sire                     │ 5658                       │
├──────────────────────────────────┼────────────────────────────┤
│ Alchemical Hydra                 │ 363                        │
├──────────────────────────────────┼────────────────────────────┤
│ Barrows Chests                   │ 137                        │
├──────────────────────────────────┼────────────────────────────┤
│ Bryophyta                        │ 5                          │
├──────────────────────────────────┼────────────────────────────┤
│ Callisto                         │ 2369                       │
├──────────────────────────────────┼────────────────────────────┤
│ Cerberus

Next, let's look instead at just finding the players with the highest kill count for each boss.

In [43]:
col_list = ['username'] + boss_list
boss_max_kc = df.filter(col_list)
boss_max_kc = boss_max_kc.set_index('username')
boss_max_kc = boss_max_kc.agg(['idxmax','max'])
boss_max_kc.columns = boss_max_kc.columns.str.replace('_score','')
boss_max_kc.columns = boss_max_kc.columns.str.replace('_',' ')
boss_max_kc= np.transpose(boss_max_kc).reset_index()
boss_max_kc['index'] = boss_max_kc['index'].str.title()
print(tabulate(boss_max_kc, headers=['Boss','Username','Kill Count'], tablefmt='fancy_grid', showindex=False))

╒══════════════════════════════════╤══════════════╤══════════════╕
│ Boss                             │ Username     │   Kill Count │
╞══════════════════════════════════╪══════════════╪══════════════╡
│ Abyssal Sire                     │ Abyssal Sire │        16675 │
├──────────────────────────────────┼──────────────┼──────────────┤
│ Alchemical Hydra                 │ Sulll        │        53134 │
├──────────────────────────────────┼──────────────┼──────────────┤
│ Barrows Chests                   │ cups ftw     │        17634 │
├──────────────────────────────────┼──────────────┼──────────────┤
│ Bryophyta                        │ tannerdino   │         5919 │
├──────────────────────────────────┼──────────────┼──────────────┤
│ Callisto                         │ Shewdoh      │        14300 │
├──────────────────────────────────┼──────────────┼──────────────┤
│ Cerberus                         │ Jacobs       │       123458 │
├──────────────────────────────────┼──────────────┼───────────